### This Notebook contains the details of the visualization and analyis of the Popular electric vehcicle data set from the state.
#### Data Visualizations and Regression.

##### Step 1:  Import Required libraries


In [48]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show    
from bokeh.io import output_notebook       
from bokeh.models import ColumnDataSource  # We will need this when preparing our data for a bar/column plot

import matplotlib.pyplot as plt
import seaborn as sns


output_notebook()  

Loading BokehJS ...

In [49]:
#Ref Dataset: https://catalog.data.gov/dataset/electric-vehicle-population-data
df = pd.read_csv(r'Electric_Vehicle_Population_Data.csv')
df.head(2)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJYGDEE1L,King,Seattle,WA,98122.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37.0,125701579,POINT (-122.30839 47.610365),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
1,7SAYGDEE9P,Snohomish,Bothell,WA,98021.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,1.0,244285107,POINT (-122.179458 47.802589),PUGET SOUND ENERGY INC,5.306105e+10


#### Step 2: Data Cleaning

In [5]:
#View columns and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         177866 non-null  object 
 1   County                                             177861 non-null  object 
 2   City                                               177861 non-null  object 
 3   State                                              177866 non-null  object 
 4   Postal Code                                        177861 non-null  float64
 5   Model Year                                         177866 non-null  int64  
 6   Make                                               177866 non-null  object 
 7   Model                                              177866 non-null  object 
 8   Electric Vehicle Type                              177866 non-null  object

In [50]:
print("Null count per column:")
for column in df.columns:
    null_count = df[column].isnull().sum()
    print(f"{column}: {null_count}")

Null count per column:
VIN (1-10): 0
County: 5
City: 5
State: 0
Postal Code: 5
Model Year: 0
Make: 0
Model: 0
Electric Vehicle Type: 0
Clean Alternative Fuel Vehicle (CAFV) Eligibility: 0
Electric Range: 0
Base MSRP: 0
Legislative District: 389
DOL Vehicle ID: 0
Vehicle Location: 9
Electric Utility: 5
2020 Census Tract: 5


In [51]:
df = df.dropna(subset=['Legislative District', 'Vehicle Location'])
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 177473 entries, 0 to 177865
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         177473 non-null  object 
 1   County                                             177473 non-null  object 
 2   City                                               177473 non-null  object 
 3   State                                              177473 non-null  object 
 4   Postal Code                                        177473 non-null  float64
 5   Model Year                                         177473 non-null  int64  
 6   Make                                               177473 non-null  object 
 7   Model                                              177473 non-null  object 
 8   Electric Vehicle Type                              177473 non-null  object 
 9 

In [9]:
## Rechecking for null values again
for column in df.columns:
    null_count = df[column].isnull().sum()
    print(f"{column}: {null_count}")

VIN (1-10): 0
County: 0
City: 0
State: 0
Postal Code: 0
Model Year: 0
Make: 0
Model: 0
Electric Vehicle Type: 0
Clean Alternative Fuel Vehicle (CAFV) Eligibility: 0
Electric Range: 0
Base MSRP: 0
Legislative District: 0
DOL Vehicle ID: 0
Vehicle Location: 0
Electric Utility: 0
2020 Census Tract: 0


In [10]:
df.describe()

,Postal Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,2020 Census Tract
count,177473.000000,177473.000000,177473.000000,177473.000000,177473.000000,1.774730e+05,1.774730e+05
mean,98260.199332,2020.517115,58.826548,1070.609839,29.127710,2.202527e+08,5.303981e+10
std,303.694979,2.989179,91.973457,8346.920672,14.892259,7.585304e+07,1.620333e+07
min,98001.000000,1997.000000,0.000000,0.000000,1.000000,4.385000e+03,5.300195e+10
25%,98052.000000,2019.000000,0.000000,0.000000,18.000000,1.815241e+08,5.303301e+10
50%,98122.000000,2022.000000,0.000000,0.000000,33.000000,2.282691e+08,5.303303e+10
75%,98370.000000,2023.000000,75.000000,0.000000,42.000000,2.548504e+08,5.305307e+10
max,99403.000000,2024.000000,337.000000,845000.000000,49.000000,4.792548e+08,5.307794e+10


#### Feature Exploration and Feature engineering


In [52]:
#Examining State for unique Valaues
df['State'].unique()

array(['WA'], dtype=object)

In [53]:
# Since the state column has only one value 'WA', the column is not useful. Will drop it and move on to County
df= df.drop(['State'], axis= 1)
df['County'].unique()

array(['King', 'Snohomish', 'Kitsap', 'Thurston', 'Yakima', 'Island',
       'Chelan', 'Skagit', 'Spokane', 'Walla Walla', 'Grant', 'Stevens',
       'Whitman', 'Kittitas', 'Pend Oreille', 'Clark', 'Cowlitz',
       'Jefferson', 'Clallam', 'Klickitat', 'Pierce', 'Grays Harbor',
       'Whatcom', 'Skamania', 'Wahkiakum', 'Benton', 'Franklin', 'Lewis',
       'San Juan', 'Mason', 'Okanogan', 'Douglas', 'Adams', 'Pacific',
       'Asotin', 'Columbia', 'Lincoln', 'Ferry', 'Garfield'], dtype=object)

There are 2 more columns that point to Location: City, and State. So will create a new column Location which is a concatenation of  City and State columns, with each value separated by a comma.
df['Location'] = df['County'] + ', ' + df['City'] + ', ' + df['State']

In [75]:
df['Location'] = df['County']+ ', ' + df['City']

len (df['Location'].unique())

511

In [77]:
df.columns


Index(['VIN (1-10)', 'County', 'City', 'Postal Code', 'Model Year', 'Make',
       'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract',
       'Electric_Range_Category', 'Location'],
      dtype='object')

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177473 entries, 0 to 177865
Data columns (total 18 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         177473 non-null  object 
 1   County                                             177473 non-null  object 
 2   City                                               177473 non-null  object 
 3   Postal Code                                        177473 non-null  float64
 4   Model Year                                         177473 non-null  int64  
 5   Make                                               177473 non-null  object 
 6   Model                                              177473 non-null  object 
 7   Electric Vehicle Type                              177473 non-null  object 
 8   Clean Alternative Fuel Vehicle (CAFV) Eligibility  177473 non-null  object 
 9 

In [84]:
cols_to_check = ['Electric Vehicle Type', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility']

# Print unique values for each column using a for loop
for column in cols_to_check:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'Electric Vehicle Type': ['Battery Electric Vehicle (BEV)' 'Plug-in Hybrid Electric Vehicle (PHEV)']
Unique values in column 'Clean Alternative Fuel Vehicle (CAFV) Eligibility': ['Clean Alternative Fuel Vehicle Eligible'
 'Eligibility unknown as battery range has not been researched'
 'Not eligible due to low battery range']


In [85]:
make_groups = df.groupby('Make') # Creates a grouping of the DataFrame by column 'make'
EVType_groups = df.groupby('Electric Vehicle Type') # Creates a grouping of the df by EV tyoe
EVCAF_groups = df.groupby('Clean Alternative Fuel Vehicle (CAFV) Eligibility')

Creating Electric ange groups based on Base MSRP values

In [86]:
df['Base MSRP'].value_counts()

Base MSRP
0         174142
69900       1366
31950        381
52900        222
32250        136
59900        127
54950        126
39995        111
36900        104
44100         91
45600         82
64950         81
33950         76
52650         68
34995         65
36800         55
55700         54
53400         37
81100         23
110950        20
98950         19
90700         18
75095         13
102000        12
184400        10
43700         10
89100          8
109000         7
91250          5
32995          3
845000         1
Name: count, dtype: int64

The 'Electric_Range_Category' column has a range of valuees so will group based on observations.

"Unknown": If the 'Electric Range' value is 0, we assign this category as it might indicate missing or unknown values.
"Short": If the 'Electric Range' value is less than 150, we assign this category.
"Medium": If the 'Electric Range' value is between 150 and 300, we assign this category.
"Long": If the 'Electric Range' value is greater than 300, we assign this category.
By creating this new column, we can better understand the distribution of electric vehicle ranges in our dataset and account for the potential uncertainty introduced by the large number of 0 values in the 'Electric Range' column.

In [87]:
def create_electric_range_category(df, column='Electric Range'):
    def categorize_range(electric_range):
        if electric_range == 0:
            return "Unknown"
        elif electric_range < 150:
            return "Short"
        elif electric_range < 300:
            return "Medium"
        else:
            return "Long"

    df['Electric_Range_Category'] = df[column].apply(categorize_range)
    return df

df = create_electric_range_category(df, column='Electric Range')
# Display new df with Electric range category
#df['Electric_Range_category'].value_counts()
df.head()

,VIN (1-10),County,City,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract,Electric_Range_Category,Location
0,5YJYGDEE1L,King,Seattle,98122.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37.0,125701579,POINT (-122.30839 47.610365),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10,Medium,"King, Seattle"
1,7SAYGDEE9P,Snohomish,Bothell,98021.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,1.0,244285107,POINT (-122.179458 47.802589),PUGET SOUND ENERGY INC,5.306105e+10,Unknown,"Snohomish, Bothell"
2,5YJSA1E4XK,King,Seattle,98109.0,2019,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,270,0,36.0,156773144,POINT (-122.34848 47.632405),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10,Medium,"King, Seattle"
3,5YJSA1E27G,King,Issaquah,98027.0,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,5.0,165103011,POINT (-122.03646 47.534065),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10,Medium,"King, Issaquah"
4,5YJYGDEE5M,Kitsap,Suquamish,98392.0,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,23.0,205138552,POINT (-122.55717 47.733415),PUGET SOUND ENERGY INC,5.303594e+10,Unknown,"Kitsap, Suquamish"


In [88]:
##Creating rgroups of car makes
#FORD_data = groups.get_group('FORD')
#FORD_data.head()

Step 4: Visualizations with Bokeh

##### Top Electric Vehicles by distribution

In [90]:
#Import Bokeh libraries
from bokeh.transform import factor_cmap
from bokeh.palettes import Viridis256
from bokeh.palettes import Bright6

# Count the number of electric vehicles by make and get the top 12 makes
ev_counts_by_make = df['Make'].value_counts().nlargest(12)
ev_counts_by_make = ev_counts_by_make[::-1] #To ensure plots are sorted from highest to lowest

# Creating Bokeh figure
p = figure(y_range=list(ev_counts_by_make.index), height=400, width=600, title="Top 10 Electric Vehicle Makes by Number of Electric Vehicles",
           toolbar_location=None, tools="")

source = ColumnDataSource(data=dict(make=list(ev_counts_by_make.index), counts=ev_counts_by_make.values))

# Define a smaller palette with 10 colors
palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Plotting horizontal bars for each make
p.hbar(y='make', right='counts', height=0.8, source=source, line_color="white",
       fill_color=factor_cmap('make', palette=palette, factors=list(ev_counts_by_make.index)))

# Customize plot with labels
p.xaxis.axis_label = "Number of Vehicles"
p.yaxis.axis_label = "Make"

# Show the plot
show(p)

##### We can see clearly that Tesla has the highest distribution onf electric vehicles in the state of California

Trend of Electric Vehicle over the years

In [20]:
df.head(1)

,VIN (1-10),County,City,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract,Electric_Range_Category
0,5YJYGDEE1L,King,Seattle,98122.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37.0,125701579,POINT (-122.30839 47.610365),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10,Medium


In [23]:
len(df['DOL Vehicle ID'].unique())
#This is the unique Identifier column in the data

177473

In [29]:
ev_counts = df['DOL Vehicle ID'].count() # The count of unique EVs
ev_counts

177473

In [70]:


# Calculate the count of unique EVs per year
ev_counts = df['Model Year'].value_counts().sort_index()

# Convert 'Model Year' index to string for Bokeh interpretation
Year = ev_counts.index.astype(str)

# Create a ColumnDataSource
source = ColumnDataSource(data=dict(Year=Year, ev_counts=ev_counts))

# Create Bokeh figure
visual = figure(height=600, width=600, title="Yearly Distribution of EV vehicles",
                x_axis_label="Year", y_axis_label="Count of EVs")

# Plot line chart
visual.line(x='Year', y='ev_counts', line_width=2, source=source)

# Show the plot
show(visual)


In [91]:
# Count the number of electric vehicles by make and get the top 10 makes
ev_counts_by_location = df['Location'].value_counts().nlargest(12)
ev_counts_by_location = ev_counts_by_location[::-1] #To ensure plots are sorted from highest to lowest

# Creating Bokeh figure
p = figure(y_range=list(ev_counts_by_location.index), height=400, width=600, 
                     title="Top 12 Locations with the highest Number of Electric Vehicles",
           toolbar_location=None, tools="")

source = ColumnDataSource(data=dict(make=list(ev_counts_by_location.index), counts=ev_counts_by_location.values))

# Define a smaller palette with 10 colors
colors = ['#256676', '#5cdac5', '#277a35', '#70de63', '#333a9e', '#e057e1', '#d0bcfe', '#6a7fd2', '#99ceeb', '#6a10a6', '#991c64', '#573f56']

# Plotting horizontal bars for each make
p.hbar(y='make', right='counts', height=0.8, source=source, line_color="white", fill_color=factor_cmap('make', palette=colors,
               factors=list(ev_counts_by_make.index)))

# Customize plot with labels
p.xaxis.axis_label = "Number of Vehicles"
p.yaxis.axis_label = "Make"

# Show the plot
show(p)

In [73]:
# Plotting vertical bars for each make
p.vbar(x='make', top='counts', width=0.8, source=source, line_color="white", fill_color=factor_cmap('make', palette=colors,
               factors=list(ev_counts_by_location.index)))

# Customize plot with labels
p.yaxis.axis_label = "Number of Vehicles"
p.xaxis.axis_label = "Make"

# Rotate x-axis labels
#p.xaxis.major_label_orientation = pi / 4

# Show the plot
show(p)
